In [44]:
W = [
    [[0, 16, 16, 10],
     [0,  0,  8,  8],
     [0,  0,  0,  4],
     [0,  0,  0,  0]],
    
    [[0, 5, 4,  7, 6,  8],
     [0, 0, 7, 10, 9, 11],
     [0, 0, 0,  7, 6,  8],
     [0, 0, 0,  0, 5,  9],
     [0, 0, 0,  0, 0,  8],
     [0, 0, 0,  0, 0,  0]],
]

N = [
    'ABCD',
    'ABCDEF',
]

In [45]:
class node:
    name = ''
    final = True
    node_dist = 0
    child_nodes = []
    
    def __init__(self, name, final=True, child_node_1=None, child_node_2=None, node_dist_1=0, node_dist_2=0):
        self.name = name
        self.final = final
        
        if not final:
            child_node_1.node_dist = node_dist_1
            child_node_2.node_dist = node_dist_2
            self.child_nodes = [child_node_1, child_node_2]
    
    def __str__(self):
        if self.final:
            return self.name + ':' + str(round(self.node_dist, 2))
        else:
            str_child_1 = str(self.child_nodes[0])
            str_child_2 = str(self.child_nodes[1])
            if self.node_dist != 0:
                return '(' + str_child_1 + ',' + str_child_2 + '):' + str(round(self.node_dist, 2))
            else:
                return '(' + str_child_1 + ',' + str_child_2 + ')'

def dist(WM, for_i, except_j):
    L = len(WM)
    
    dist = 0
    
    for i in range(for_i):
        dist += WM[i][for_i]
    
    for j in range(for_i + 1, L):
        if EXCLUDE_B:
            if j != except_j:
                dist += WM[for_i][j]
        else:
            dist += WM[for_i][j]
    
    return dist / (L - 2)
            
def find_min(WM):
    L = len(WM)
    
    cur_i = 0
    cur_j = 1
    dist1 = dist(WM, cur_i, cur_j)
    dist2 = dist(WM, cur_j, cur_i)
    cur_min = WM[cur_i][cur_j] - dist1 - dist2

    for i in range(L):
        for j in range(i + 1, L):
            dist1 = dist(WM, i, j)
            dist2 = dist(WM, j, i)
            new_dist = WM[i][j] - dist1 - dist2
            
            if new_dist < cur_min:
                cur_i = i
                cur_j = j
                cur_min = new_dist
    return cur_i, cur_j

def NJ(weight_matrix, names):
    res = [node(n) for n in names]

    while len(weight_matrix) > 2:
        i, j = find_min(weight_matrix)

#         full_dist = r_min / 2
        node_dist_1 = 0.5 * (weight_matrix[i][j] + dist(weight_matrix, i, j) - dist(weight_matrix, j, i))
        node_dist_2 = 0.5 * (weight_matrix[i][j] + dist(weight_matrix, j, i) - dist(weight_matrix, i, j))
            
        names[i] = names[i] + names[j]
        names.pop(j)
        
        res[i] = node(names[i], False, res[i], res[j], node_dist_1, node_dist_2)
        res.pop(j)
        
        L = len(weight_matrix)
        
        D_a_b = weight_matrix[i][j]
        
        
        # update i row
        for k in range(i + 1, L):
            if k != j:
                temp_w = 0
                if k < j:
                    temp_w = weight_matrix[k][j]
                else:
                    temp_w = weight_matrix[j][k]
                                
                weight_matrix[i][k] = (weight_matrix[i][k] + temp_w - D_a_b) / 2
        
        # update i column
        for k in range(0, i):
            weight_matrix[k][i] = (weight_matrix[k][i] + weight_matrix[k][j] - D_a_b) / 2
        
        for k in range(L):
            weight_matrix[k].pop(j)
        
        weight_matrix.pop(j)
    
    res[1].node_dist = weight_matrix[0][-1]
    print(res[0])
    print(res[1])

def start_test(test):
    names_test = [i for i in N[test]]
    weight_matrix_test = [w.copy() for w in W[test]]

    NJ(weight_matrix_test, names_test)

В конспекте лекции указано, что среднее расстояние от A до всех остальных, может считаться включая B и не включая его. Посольку результаты получаются разные и результаи не включая В в чем-то ближе к истинному дереву, я добавил опцию EXCLUDE_B, в зависимости от которой В исключается при расчете расстояния (EXCLUDE_B = True) или включается (EXCLUDE_B = False).

Также в задании требуется вывести дерево в формате Newick, однако так как в результате NJ алгоритма дерево получается неукорененным, сделать это для всего дерева невозможно, результаты выводятся для двух вершин на двух строчках, расстояние между ними показывается после двоеточия на второй строчке (присваивается второй выведенной вершине).

Test 1.

In [46]:
test = int(input()) - 1
EXCLUDE_B = True
start_test(test)

1
A:0
(B:2.5,(C:2.5,D:1.5):3.5):10.5


In [47]:
test = int(input()) - 1
EXCLUDE_B = False
start_test(test)

1
((A:10.0,D:0.0):2.0,B:5.0)
C:3.0


Test 2.

In [48]:
test = int(input()) - 1
EXCLUDE_B = True
start_test(test)

2
(((A:0.38,B:4.62):0.5,C:2.5),(D:1.75,E:3.25):2.0)
F:5.0


In [49]:
test = int(input()) - 1
EXCLUDE_B = False
start_test(test)

2
((((A:1.0,B:4.0):1.0,C:2.0):1.0,F:5.0):1.0,D:3.0)
E:2.0
